In [1]:
############# libraries ###########


import pandas as pd
import psycopg2
import pygsheets
import datetime as dt
from datetime import date, timedelta,datetime
import calendar
import numpy as np
import sys
from pyhive import presto
from sqlalchemy import create_engine
import ast
import time
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
import requests
import json
#from pandas.io.json import json_normalize
import oyoms as om
from oyoms import WorkbookClient
from azure.storage.blob import BlobServiceClient
from azure.identity import DefaultAzureCredential
from tempfile import NamedTemporaryFile

In [2]:

session = requests.Session()
session.headers.update({'X-Trino-Password': '123456789'})

########## connection #############

try:
    conn_presto = presto.Connection(host="az-presto.oyorooms.io", port = 8889, username="bhuvan.gogrot@oyorooms.com", requests_session=session )
except:
    conn_presto= None
    print ('connected')

In [3]:
conn_presto

In [4]:
today = datetime.utcnow()
week_before_today = datetime.utcnow() - timedelta(days=7)
last_updated_date = week_before_today.strftime('%Y-%m-%d')
formatted_date = today.strftime('%Y-%m-%d')
f_d = datetime.utcnow().strftime('%Y-%m-%d')

In [5]:
last_updated_date

'2024-06-26'

In [6]:
formatted_date

'2024-07-03'

In [7]:
brns = f"""
SELECT date(A.date1) as date,b.id as booking_id,b.invoice_no,b.check_in,b.check_out,date(created_at) as created_date,hs.oyo_id,hs.city_name,'India' as country_name,coalesce(c.prepaid_flag,0) as prepaid_flag,b.status,
case when b.source in (22,23,38) and b.micro_market_id is null then 'oyo'
                            when b.source in (17,21) and b.micro_market_id is null then 'oyo'
                            when b.source in (0,7)  and (b.micro_market_id is null OR b.micro_market_id = 99) then 'oyo'
                            when b.source = 4  and b.micro_market_id is null then 'oyo'
                            when b.source in (1,10,64)  and b.micro_market_id is null then 'OTA'
                            when  b.micro_market_id is not null and b.micro_market_id > 0 and b.micro_market_id != 99 then 'OBA'
                            else 'Walkin' end as booking_source, 
                            b.oyo_rooms as brns,case when b.status in (1,2)  then b.oyo_rooms else 0 end as urns,
                            gmv/(case when date_diff('day',b.check_in,b.check_out) = 0 then 1 else date_diff('day',b.check_in,b.check_out) end) as daily_gmv,
                            b.coupon_code,b.rc,created_at +interval '5' hour + interval '30' minute as lca,cancellation_time +interval '5' hour + interval '30' minute as lct,guest_id,Guest_Name,channel from 
                            
(select date_add('day',checkin,date'1970-01-01') as check_in,date_add('day',checkout,date'1970-01-01') as check_out,invoice_no,source,micro_market_id,id,after_booking_status,cancellation_reason,hotel_id,notes,status,oyo_rooms,FROM_UNIXTIME(cancellation_time/1000000) as "cancellation_time",FROM_UNIXTIME(updated_at/1000000) as "Update_Date",FROM_UNIXTIME(checkin_time/1000000) as "checkin_time",FROM_UNIXTIME(checkout_time/1000000) as "checkout_time",coupon_code,FROM_UNIXTIME(created_at/1000000) as "created_at",
--case when date(checkin) = date(checkout) then date(checkout) + interval '1' day else date(checkout) end as checkout1,
(coalesce(selling_amount,amount)-coalesce(discount,0)) as gmv,guest_id,ota_id
,case
when ota_id in (values 2,2002) then 'Agoda'
when ota_id in (values 2007) then 'EaseMyTrip'
when ota_id in (values 5,113,2005) then 'Booking.com'
when ota_id in (values 63) then 'GOMMT'
when ota_id in (values 3) then 'expedia'
when ota_id in (values 42,2048) then 'HotelBeds'
when ota_id in (values 272,30) then 'Cleartrip'
when ota_id in (values 4) then 'Yatra'
when micro_market_id is not null and micro_market_id > 0 and micro_market_id not in (306,126,256,99) then 'MM'
else 'REST'
end rc,0 as Guest_Name,split_part(external_reference_id, '_', 1) as channel
--(case when date(checkout) = date(checkin) then date(cast(checkout as date)+interval '1' day) else date(checkout) end) as checkout_v2,
--date_diff('day',cast(checkin as date),cast(checkout as date)) as r
FROM ingestiondb_service.bookings
WHERE status in (values 0,1,2,3,4,12,13) and inserted_at >= '202301' --and id= 264992750
) b 
INNER JOIN
(SELECT DISTINCT date(cal.date) as date1 FROM ingestiondb.calendar as cal
WHERE date(cal.date) BETWEEN date '2024-07-01' AND date '2024-07-31'
)A
ON A.date1 >= b.check_in and A.date1 < b.check_out
inner join 
(select hotel_id,oyo_id,city_name,state_name from aggregatedb.hotels_summary
where country_id in (values 1)
and oyo_id in ('AHM527','ZKR082','JAI890','VAD217','VAD281','MRB018'))hs
on hs.hotel_id = b.hotel_id
left join
(select distinct(p.booking_id) as booking_id, 1 as prepaid_flag
from (select id,merchant,status,is_visible,booking_id,payment_type,amount from ingestiondb_service.payment_transacs p where inserted_at >= '202301') p
where p.merchant not in (values 0,1,2,17,35,34,37,45,47,48,49,50,67,73,44,6,46,56,42,16,65,31,57) and p.status= 1 and p.is_visible = TRUE)
C ON b.id=C.booking_id
"""
# 'AHM527','ZKR082','JAI890','VAD217','VAD281','MRB018','VAD055'

In [8]:
Base = pd.read_sql(brns,conn_presto)

C:\Users\BhuvanGogrot\AppData\Local\Temp\ipykernel_17868\2159510121.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Base = pd.read_sql(brns,conn_presto)


In [9]:
excel_link = 'https://oyoenterprise-my.sharepoint.com/:x:/r/personal/bhuvan_gogrot_oyorooms_com/Documents/Old%20Query.xlsx?d=w581cb3f16aa94e54bb2fbeb3569689fe&csf=1&web=1&e=XjaVVQ'
wb = WorkbookClient('bhuvan.gogrot@oyorooms.com', excel_link)
 
try:
    wb.clear_range('Base', range_address='A2:H30000')
except:
    print('No blank rows')
 
batchsize = 8000
row_num = 2
for i in range(0, len(Base), batchsize):
    print(i,i+batchsize,'B'+str(row_num))
    try:
        wb.write_data('Base', Base[i:i+batchsize], location='A'+str(row_num),include_header=False,ignore_timeout=True)
    except:
        print('Continue')
    row_num+=batchsize

0 8000 B2
